In [1]:
import os
from tqdm import tqdm
from itertools import chain
import pandas as pd
import numpy as np
import random, torch
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'white', font_scale=1.5)
import sys

sys.path.insert(0,  '/home/was966/Research/mims-compass/')
from compass.utils import plot_embed_with_label
from compass import PreTrainer, FineTuner, loadcompass #, get_minmal_epoch
from compass.utils import plot_embed_with_label, plot_performance, score2
from compass.tokenizer import CANCER_CODE
def onehot(S):
    assert type(S) == pd.Series, 'Input type should be pd.Series'
    dfd = pd.get_dummies(S, dummy_na=True)
    nanidx = dfd[dfd[np.nan].astype(bool)].index
    dfd.loc[nanidx, :] = np.nan
    dfd = dfd.drop(columns=[np.nan])*1.
    cols = dfd.sum().sort_values(ascending=False).index.tolist()
    dfd = dfd[cols]
    return dfd

In [5]:
df = pd.read_pickle('/home/was966/Research/hms-work-data/TCGA-ITRP-GENESET-MERGE/15672/RESPONSE.ALL.V123456789.TABLE')
genes = df.columns[:-4]

on = df[df.timepoint == 'On']

dfl2 = on[on.columns[-4:]]
dfl2 = dfl2[dfl2.cancer_type == 'GBM']

dfl2 = dfl2.rename(columns = {'response':'response_label', 'timepoint':'Timing'})
dfl2['Timing'] = 'Post'
dfl2['Therapy'] = 'PD1' ## GBM cohort is PD1 in the paper

dft2 = on[genes].loc[dfl2.index]


In [6]:
data_path = '/home/was966/Research/hms-work-data/PSOG.bak/paper/data_processing/ITRP_dataset_processing/version2/on_treatment/'

dft1 = pd.read_excel(os.path.join(data_path, 'v2.tpm.xlsx'), index_col=0)
dft1 = dft1.loc[genes].T
dft1.index = dft1.index + '@' + 'On'
dfl1 = pd.read_excel(os.path.join(data_path, 'v2.samples.xlsx'), index_col=0)

dfl1.index = dfl1.index + '@' + 'On'

dft1 = dft1.loc[dfl1.index]

dfl1 = dfl1.rename(columns = {'Freeman_response':'response_label', })
dfl1['cancer_type'] = 'SKCM'

In [161]:
dfl1

,Sample_id,RECIST,Patient,Age,Gender,Primary_tumor,Overall_survival,"Alive (Y=1, No=0)",Timing,response_label,Therapy,Prior_CTLA4,cohort,tmb.dan,tmb.wang,participant,is_in_FreeMan,rnaseq_id,cancer_type
rna_id,,,,,,,,,,,,,,,,,,,
107T-Tumor-SM-BZ9WF@On,107CR,NaN,107T,63.0,F,Uveal,998.0,0.0,Post,NR,PD1,1.0,MGH,NaN,NaN,107T,False,NaN,SKCM
115T-Tumor-SM-BZ9WH@On,115BR,NaN,115T,66.0,M,Skin,1092.0,1.0,Post,NR,PD1,1.0,MGH,NaN,NaN,115T,False,NaN,SKCM
122T-Tumor-SM-BZ9WU@On,122BR,NaN,122T,79.0,M,Skin,1061.0,1.0,Post,R,PD1,1.0,MGH,NaN,NaN,122T,False,NaN,SKCM
148T-Tumor-SM-AH3NK@On,148BR,NaN,148T,81.0,M,Skin,420.0,0.0,Post,NR,PD1+KIR,0.0,MGH,NaN,NaN,148T,False,NaN,SKCM
155T-Tumor-SM-BZ9WW@On,155BR,NaN,155T,63.0,M,Uveal,278.0,0.0,Post,NR,CTLA4,1.0,MGH,NaN,NaN,155T,False,NaN,SKCM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36_ipiPD1_EDT@On,ipiPD1_36_EDT,NaN,36_ipiPD1,NaN,NaN,NaN,637.0,1.0,Post,R,CTLA4+PD1,1.0,Gide,NaN,NaN,36_ipiPD1,False,ERR2208913,SKCM
41_PD1_EDT@On,PD1_41_EDT,NaN,41_PD1,NaN,NaN,NaN,886.0,1.0,Post,R,PD1,NaN,Gide,NaN,NaN,41_PD1,False,ERR2208957,SKCM
47_PD1_EDT@On,PD1_47_EDT,NaN,47_PD1,NaN,NaN,NaN,1174.0,1.0,Post,R,PD1,NaN,Gide,NaN,NaN,47_PD1,False,ERR2208963,SKCM


In [7]:
dfl = dfl2._append(dfl1)
dft = dft2._append(dft1)


dfl = dfl[~dfl.response_label.isna()]
dft = dft.loc[dfl.index]

In [8]:
dft.to_pickle('POST_TREATMENT_TPM.TABLE')

In [9]:
dfl.to_pickle('POST_TREATMENT_SAMPLE.TABLE')

In [10]:
dfl.groupby('cohort').size()

cohort
Gide              16
MGH               48
Prins_GBM_2019    14
Riaz              56
Zhao               9
dtype: int64

In [317]:
df_label = pd.read_pickle('POST_TREATMENT_SAMPLE.TABLE')
df_tpm = pd.read_pickle('POST_TREATMENT_TPM.TABLE')

## Keep only PD1 and CTLA4
df_label = df_label[(df_label.Therapy.isin(['PD1', 'CTLA4', 'CTLA4+PD1'])) & df_label.cohort.isin(['Riaz', 'Gide', 'MGH'])] #'Prins_GBM_2019', 'Zhao' # 'CTLA4+PD1',  
df_tpm = df_tpm.loc[df_label.index]
len(df_label)

111

In [318]:
# df_label = df_label[df_label.Timing == 'Post']
# df_tpm = df_tpm.loc[df_label.index]

In [351]:
df_label1 = pd.read_pickle('/home/was966/Research/mims-compass/paper//00_data/ITRP.PATIENT.TABLE')
df_tpm1 = pd.read_pickle('/home/was966/Research/mims-compass/paper//00_data/ITRP.TPM.TABLE')

paired_patient = df_label1[df_label1.patient_id.isin(df_label.Patient)].patient_id

Pre_samples = df_label1[df_label1.patient_id.isin(paired_patient)].sort_values('patient_id')
Pre_samples['Therapy'] = Pre_samples['ICI_target_map'].apply(lambda x:'+'.join(pd.Series(x.split('+')).sort_values().tolist()))

Post_samples = df_label[df_label.Patient.isin(paired_patient)].sort_values('Patient')
Post_samples['patient_id'] = Post_samples['Patient']
Post_samples['Therapy'] = Post_samples['Therapy'].apply(lambda x:'+'.join(pd.Series(x.split('+')).sort_values().tolist()))

print(len(Post_samples), len(Pre_samples))

93 81


In [352]:
Post_matched = []
Pre_matched = []

for i in range(len(Pre_samples)):
    ts = Pre_samples.iloc[i]

    pid = ts.patient_id
    rsp = ts.response_label
    drug = ts.Therapy
    post_df = Post_samples[Post_samples.patient_id == pid]

    if ts.cohort == 'MGH':
        
        #post_df = post_df[post_df.Therapy == drug]
        post_df = post_df[post_df.response_label== rsp] ## 如果label有冲突，应该以post的label为准
        post_df = post_df.sort_values('Timing')
        
        if len(post_df) >= 1:
            sidx = post_df.index.tolist()#[:1]
            Post_matched.extend(sidx)
            Pre_matched.extend([ts.name for i in range(len(sidx))])
        else:
            print(i, ts.cohort, pid)
    else:
        post_df = post_df.sort_values('Timing')
        sidx = post_df.index.tolist()#[:1]
        Post_matched.extend(sidx)
        Pre_matched.extend([ts.name for i in range(len(sidx))])
        
len(Post_matched), len(Pre_matched)

3 MGH 122T
7 MGH 166T
37 MGH 99T


(86, 86)

In [353]:
Post_samples = Post_samples.loc[Post_matched]
Pre_samples = Pre_samples.loc[Pre_matched]

Post_samples.response_label = Pre_samples.response_label.tolist()

Post_TPM = df_tpm.loc[Post_samples.index]
Pre_TPM = df_tpm1.loc[Pre_samples.index]

In [354]:
Post_samples.index = Post_samples['patient_id'] + '@' + Post_samples.reset_index().index.astype(str)
Post_TPM.index = Post_samples.index


Pre_samples.index = Pre_samples['patient_id'] + '@' + Pre_samples.reset_index().index.astype(str)
Pre_TPM.index = Pre_samples.index


In [355]:
Pre_samples.to_pickle('./PAIRED_PD1_CTLA4_PRE_SMPLE.TABLE')
Post_samples.to_pickle('./PAIRED_PD1_CTLA4_POST_SMPLE.TABLE')

Pre_TPM.to_pickle('./PAIRED_PD1_CTLA4_PRE_TPM.TABLE')
Post_TPM.to_pickle('./PAIRED_PD1_CTLA4_POST_TPM.TABLE')

In [356]:
Post_samples.response_label.value_counts()

response_label
NR    64
R     22
Name: count, dtype: int64

In [357]:
Post_samples.Therapy.value_counts()

Therapy
PD1          71
CTLA4+PD1     9
CTLA4         6
Name: count, dtype: int64

In [358]:
Post_samples.cohort.value_counts()

cohort
Riaz    43
MGH     27
Gide    16
Name: count, dtype: int64

## As one patient may have multiple post-treatment pairs, total 86 pre-post treatment pairs, invloving 78 patients, 3 cohorts

### Riaz(n=43), MGH(n=27), Gide(n=16)

In [359]:
Post_samples.patient_id.unique().shape

(78,)

In [360]:
len(Post_samples)

86